In [ ]:
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, plot_heatmaps,
    EMA, SMA, SLOPE, CHOP,
    EmpyrealOrderlySDK,
)
from emp_orderly_types import PerpetualAssetType, Interval, OrderType

load_dotenv()

In [2]:
wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [ ]:
class EMSB(Strategy):
    order_size: float = 0.5
    nine_days_length: int = 9
    fourteen_days_length: int = 14
    length: int = 14
    
    @classmethod
    def update_lags(cls, nine_days_length, fourteen_days_length, length):
        cls.nine_days_length = nine_days_length
        cls.fourteen_days_length = fourteen_days_length
        cls.length = length

    def init(self):
        close = self.data.close
        self.ema_nine_days = self.I(EMA, close, self.nine_days_length)
        self.ema_fourteen_days = self.I(EMA, close, self.fourteen_days_length)
        self.slope = self.I(SLOPE, close, self.length)

    def next(self):
        if (
            crossover(self.ema_nine_days, self.ema_fourteen_days) &
            (self.slope > 15)
        ):
            self.position.close()
            self.buy(size=self.order_size)
        elif (
            crossover(self.ema_fourteen_days, self.ema_nine_days) &
            (self.slope < -15)
        ):
            self.position.close()
            self.sell(size=self.order_size)


tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(SmaCross)
await tester.load_data(
    lookback=5,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.BTC,
)

# backtest
tester.backtest()

# plot
tester.plot(show_price_data=False)
plt.show()